#Summary Analyses

In [ ]:
dataPath = fullfile(projectRoot,'data',projectDate,subject);


% Choose the categories of neurons to compare
ddmEpoch = 'Stim';

classicEpoch = 'presaccPeak';
cancelEpoch = 'presaccPeakCancel';

classicEpoch = 'presaccRamp';
cancelEpoch = 'presaccRampCancel';

classicEpoch = 'presacc';
cancelEpoch = 'presaccCancel';




% LOAD ALL RELEVANT DATA AND LISTS

% Load full popuulation of neurons
load(fullfile(dataPath, ['ccm_neuronTypes', addMulti]));
nUnit = size(neuronTypes, 1);
clear neuronTypes


% Load Classic neuron classifications
c = load(fullfile(dataPath, ['ccm_',classicEpoch,'_neurons', addMulti]));
classic = c.neurons;


% Load Ding and gold ddm neuron classifications
dg = load(fullfile(dataPath, ['ccm_ddm', ddmEpoch, '_neurons', addMulti]));
ddm = dg.neurons;


% Load Cancel Time population 
ca = load(fullfile(dataPath, ['ccm_',cancelEpoch,'_neurons', addMulti]));
cancel = ca.neurons;



[~, iCla, iDdm] = setxor(classic, ddm);
classicDdm = intersect(classic, ddm);
classicNoDdm = classic(iCla, :);
ddmNoClassic = ddm(iDdm, :);


[~, iCla, iCan] = setxor(classic, cancel);
classicCancel = intersect(classic, cancel);
classicNoCancel = classic(iCla, :);
cancelNoClassic = ddm(iCan, :);


[~, iCla, iBoth] = setxor(cancel, classicDdm);
classicCancelNoDdm = cancel(iCla,:);
classicDdmNoCancel = classicDdm(iBoth,:);
classicDdmCancel = intersect(classicDdm, cancel);



###Print out percentages of the relevant neuron categories

In [ ]:
fprintf('\nTotal Units:\t%d\n', nUnit)

fprintf('\n%s:\t%d\t%.1f%% of total\n', classicEpoch, size(classic, 1), size(classic, 1)*100/nUnit)

fprintf('DDM-%s:\t%d\t%.1f%% of total\n', ddmEpoch, size(ddm, 1), size(ddm, 1)*100/nUnit)

fprintf('%s and DDM-%s:\t%d\t%.1f%% of total,\t%.1f%% of %s,\t%.1f%% of DDM-%s\n', classicEpoch, ddmEpoch, size(classicDdm, 1), size(classicDdm, 1)*100/nUnit, size(classicDdm, 1)*100/size(classic, 1), classicEpoch, size(classicDdm, 1)*100/size(ddm, 1), ddmEpoch)

fprintf('%s and %s:\t%d\t%.1f%% of total,\t%.1f%% of %s\n', classicEpoch, cancelEpoch, size(classicCancel, 1), size(classicCancel, 1)*100/nUnit, size(classicCancel, 1)*100/size(classic, 1), classicEpoch)

fprintf('%s and %s and DDM-%s:\t%d\t%.1f%% of total,\t%.1f%% of %s,\t%.1f%% of DDM-%s\n', classicEpoch, cancelEpoch, ddmEpoch, size(classicDdmCancel, 1), size(classicDdmCancel, 1)*100/nUnit, size(classicDdmCancel, 1)*100/size(classic, 1), classicEpoch, size(classicDdmCancel, 1)*100/size(ddm, 1), ddmEpoch)


## Make a Venn diagram of the results

In [ ]:
A = [size(classic, 1) size(cancel, 1) size(ddm, 1)];
I = [size(classicCancel, 1) size(classicDdm, 1) size(classicDdmCancel, 1) size(classicDdmCancel, 1)];

figure(1), axis equal, axis off

venn(A,I,'FaceColor',{'r','y','b'},'FaceAlpha',{.5,0.6,0.2},'EdgeColor','black')
